In [2]:
## le -q signifie 'quiet' et évite les messages désagréables
pip install -q geopy

SyntaxError: invalid syntax (1948582872.py, line 2)

In [3]:
from geopy.distance import geodesic
import plotly.express as px
import pandas as pd
import numpy as np
from pprint import pprint
from pyroutelib3 import Router

In [4]:
## on importe le module CarNetwork, pour tester si on a bien internalisé toutes les taches dans la classe 

import os
import sys 

car_network_directory = '/Users/khelifanail/Documents/GitHub/Pycar/Modules'

## si Augustin utilise le chemin absolu est '/Users/augustincablant/Documents/GitHub/Pycar/Modules' 

sys.path.append(car_network_directory)

from CarNetwork import CarNetwork

In [6]:
URL = 'https://www.data.gouv.fr/fr/datasets/r/517258d5-aee7-4fa4-ac02-bd83ede23d25'
df = pd.read_csv(URL, sep = ';')

In [7]:
df.columns

Index(['n_amenageur', 'n_operateur', 'n_enseigne', 'id_station', 'n_station',
       'ad_station', 'code_insee', 'xlongitude', 'ylatitude', 'nbre_pdc',
       'id_pdc', 'puiss_max', 'type_prise', 'acces_recharge', 'accessibilite',
       'observations', 'date_maj', 'source', 'geo_point_borne',
       'code_insee_commune', 'region', 'departement'],
      dtype='object')

In [8]:
def clean_df(dataframe):
    liste = []
    for row in dataframe.itertuples():
        if row.xlongitude > 90 or row.ylatitude > 90:
            liste.append(row.Index)
    return dataframe.drop(liste)

df1 = clean_df(df)        

In [10]:
df1.head(3)

,n_amenageur,n_operateur,n_enseigne,id_station,n_station,ad_station,code_insee,xlongitude,ylatitude,nbre_pdc,...,type_prise,acces_recharge,accessibilite,observations,date_maj,source,geo_point_borne,code_insee_commune,region,departement
0,ABVV Volvo Roissy,DRIVECO,ABVV Volvo Roissy,FR*SSD*PABVVVOLVO95700*1,ABVV Volvo Roissy,1 Avenue Charles de Gaulle 95700 Roissy-en-France,95527.0,2.511095,48.999531,6.0,...,T2,Gratuit,24/24 7/7 jours,NaN,2021-03-03,https://www.data.gouv.fr/fr/datasets/liste-des...,"48.999531,2.511095",95527,Île-de-France,Val-d'Oise
1,ABVV Volvo St Ouen l'Aumône,DRIVECO,ABVV Volvo St Ouen l'Aumône,FR*SSD*PABVVVOLVO95310*1,ABVV Volvo St Ouen l'Aumône,135 Rue de Paris 95310 Saint-Ouen-l'Aumône,95572.0,2.122450,49.035994,12.0,...,T2,Gratuit,24/24 7/7 jours,NaN,2021-03-03,https://www.data.gouv.fr/fr/datasets/liste-des...,"49.035994,2.12245",95572,Île-de-France,Val-d'Oise
2,ABVV Volvo St Ouen l'Aumône,DRIVECO,ABVV Volvo St Ouen l'Aumône,FR*SSD*PABVVVOLVO95310*1,ABVV Volvo St Ouen l'Aumône,135 Rue de Paris 95310 Saint-Ouen-l'Aumône,95572.0,2.122450,49.035994,12.0,...,"EF, T2",Gratuit,24/24 7/7 jours,NaN,2021-03-03,https://www.data.gouv.fr/fr/datasets/liste-des...,"49.035994,2.12245",95572,Île-de-France,Val-d'Oise


In [49]:
df_code = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25')


In [50]:
df_code.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [51]:
def coordonnée_avec_cp(code_postale):
    for row in df_code.itertuples():
        cp = row.code_postal
        if code_postale == cp:
            lat = row.latitude
            lon = row.longitude
            return [lat, lon]
    return None

In [36]:
def recherche_station_proche(dist, lat, lon):
    """ 
    Recherche la station la plus proche dans un périmètre de rayon dist et de centre de coordonnées lat, lon
    """
    lat, lon = np.float(lat), np.float(lon)
    lowest = [0, dist+10]  # index, min distance
    dataframe0 = df1.dropna(subset = ['xlongitude', 'ylatitude'])
    index = 0
    for row in dataframe0.itertuples():
        lat_s = row.ylatitude
        lon_s = row.xlongitude
        d = geodesic((lat, lon), (lat_s, lon_s)).km
        if lowest[1] > d:
            lowest[1] = d
            lowest[0] = index
        index += 1

    if lowest[1] > dist:
        return None
    else:
        index = lowest[0]
        ligne = dataframe0.iloc[[index]]
        lat = ligne['ylatitude'].values[0]
        lon = ligne['xlongitude'].values[0]
        nom = ligne['n_enseigne'].values[0]
        return lat, lon, nom

In [37]:
#exemple 
recherche_station_proche(100, 48.999535, 2.511095)

/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_4884/2453712820.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lat, lon = np.float(lat), np.float(lon)


(48.999531, 2.511095, 'ABVV Volvo Roissy')

In [38]:
def trajet(départ, fin):
    """ 
    Retourne une liste de longitude et 
    de latitude correspondant à la route à suivre pour 
    rejoindre fin depuis départ définis à partir de leur coordonées respectives
    """
    Trajet_lat = []
    Trajet_lon = []
    total = 0
    router = Router('car')
    start = router.findNode(départ[0], départ[1])
    end = router.findNode(fin[0], fin[1])
    status, route = router.doRoute(start, end)
    if status == 'success':
        routeLatLons = list(map(router.nodeLatLon, route))
        for elem in routeLatLons:
            lat, lon = elem
            Trajet_lat.append(lat)
            Trajet_lon.append(lon)
    for i in range(len(Trajet_lat)-1):
        lat_i = Trajet_lat[i]
        lat_f = Trajet_lat[i+1]
        lon_i = Trajet_lon[i]
        lon_f = Trajet_lon[i+1]
        d = geodesic((lat_i, lon_i), (lat_f, lon_f)).km
        total += d
    return Trajet_lat, Trajet_lon, total

In [42]:
def trajet_electrique(Trajet_thermique_lat, Trajet_thermique_lon, autonomie):
    Trajet_lat = []
    Trajet_lon = []
    parcourue = 0
    total = 0
    router = Router('car')
    for k in range(len(Trajet_thermique_lat)-1):
        if parcourue <= autonomie*0.7:
            lat_i = Trajet_thermique_lat[k]
            lat_f = Trajet_thermique_lat[k+1]
            lon_i = Trajet_thermique_lon[k]
            lon_f = Trajet_thermique_lon[k+1]

            start = router.findNode(lat_i, lon_i)
            end = router.findNode(lat_f, lon_f)
            status, route = router.doRoute(start, end)
            if status == 'success':
                routeLatLons = list(map(router.nodeLatLon, route))
                for i in range(len(routeLatLons)-1):
                    dep_lat = routeLatLons[i][0]
                    dep_lon = routeLatLons[i][1]
                    fin_lat = routeLatLons[i+1][0]
                    fin_lon = routeLatLons[i+1][1]
                    Trajet_lat.append(dep_lat)
                    Trajet_lat.append(fin_lat)
                    Trajet_lon.append(dep_lon)
                    Trajet_lon.append(fin_lon)
                    d = geodesic((dep_lat, dep_lon), (fin_lat, fin_lon)).km
                    parcourue += d

        else:
            station = recherche_station_proche(
                autonomie*0.3, Trajet_thermique_lat[k], Trajet_thermique_lon[k])
            if station == None:
                return None
            else:
                lat, lon, nom = station
                start = router.findNode(
                    Trajet_thermique_lat[k], Trajet_thermique_lon[k])
                end = router.findNode(lat, lon)
                status, route = router.doRoute(start, end)
                if status == 'success':
                    routeLatLons = list(map(router.nodeLatLon, route))
                    for i in range(len(routeLatLons)-1):
                        dep_lat = routeLatLons[i][0]
                        dep_lon = routeLatLons[i][1]
                        fin_lat = routeLatLons[i+1][0]
                        fin_lon = routeLatLons[i+1][1]
                        Trajet_lat.append(dep_lat)
                        Trajet_lat.append(fin_lat)
                        Trajet_lon.append(dep_lon)
                        Trajet_lon.append(fin_lon)
                        d = geodesic((dep_lat, dep_lon), (fin_lat, fin_lon)).km
                        parcourue += d
                start = router.findNode(fin_lat, fin_lon)
                end = router.findNode(
                    Trajet_thermique_lat[k], Trajet_thermique_lon[k])
                status, route = router.doRoute(start, end)
                if status == 'success':
                    routeLatLons = list(map(router.nodeLatLon, route))
                    for i in range(len(routeLatLons)-1):
                        dep_lat = routeLatLons[i][0]
                        dep_lon = routeLatLons[i][1]
                        fin_lat = routeLatLons[i+1][0]
                        fin_lon = routeLatLons[i+1][1]
                        Trajet_lat.append(dep_lat)
                        Trajet_lat.append(fin_lat)
                        Trajet_lon.append(dep_lon)
                        Trajet_lon.append(fin_lon)
                        d = geodesic((dep_lat, dep_lon), (fin_lat, fin_lon)).km
                        parcourue += d

            total += parcourue
            parcourue = 0
    total += parcourue

    return Trajet_lat, Trajet_lon, total

In [46]:
def creation_carte_autonomie(cp_départ, cp_fin, autonomie):
    départ = coordonnée_avec_cp(cp_départ)
    if départ == None:
        return 'Code Postal de départ incorrect'

    fin = coordonnée_avec_cp(cp_fin)
    if fin == None:
        return 'Code Postal de fin incorrect'

    Trajet_thermique_lat, Trajet_thermique_lon, total_thermique = trajet(
        départ, fin)
    Type = ['Thermique']*len(Trajet_thermique_lat)

    elec = trajet_electrique(
        Trajet_thermique_lat, Trajet_thermique_lon, autonomie)
    if elec == None:
        return 'Autonomie insuffisante'

    Trajet_lat_elec, Trajet_lon_elec, total_elec = elec
    Type_elec = ['Electrique']*len(Trajet_lat_elec)
    Type += Type_elec

    Trajet_lat = Trajet_thermique_lat + Trajet_lat_elec
    Trajet_lon = Trajet_thermique_lon + Trajet_lon_elec
    data = pd.DataFrame(
        {'Latitude': Trajet_lat, 'Longitude': Trajet_lon, 'Type': Type})
    fig = px.line_mapbox(data, lat='Latitude',
                         lon='Longitude',
                         color='Type', mapbox_style='open-street-map', height=800)
    return fig, total_thermique, total_elec

In [55]:
carte = creation_carte_autonomie(78670,91190,30)

/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_4884/2453712820.py:5: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_4884/2453712820.py:5: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [56]:
carte[0]

In [ ]:
def creation_carte_autonomie(cp_départ, cp_fin, autonomie):

    depart = 
    départ = coordonnée_avec_cp(cp_départ)
    if départ == None:
        return 'Code Postal de départ incorrect'

    fin = coordonnée_avec_cp(cp_fin)
    if fin == None:
        return 'Code Postal de fin incorrect'

    Trajet_thermique_lat, Trajet_thermique_lon, total_thermique = trajet(
        départ, fin)
    Type = ['Thermique']*len(Trajet_thermique_lat)

    elec = trajet_electrique(
        Trajet_thermique_lat, Trajet_thermique_lon, autonomie)
    if elec == None:
        return 'Autonomie insuffisante'

    Trajet_lat_elec, Trajet_lon_elec, total_elec = elec
    Type_elec = ['Electrique']*len(Trajet_lat_elec)
    Type += Type_elec

    Trajet_lat = Trajet_thermique_lat + Trajet_lat_elec
    Trajet_lon = Trajet_thermique_lon + Trajet_lon_elec
    data = pd.DataFrame(
        {'Latitude': Trajet_lat, 'Longitude': Trajet_lon, 'Type': Type})
    fig = px.line_mapbox(data, lat='Latitude',
                         lon='Longitude',
                         color='Type', mapbox_style='open-street-map', height=800)
    return fig, total_thermique, total_elec